In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.applications import VGG16, ResNet50
from keras.models import Sequential
from keras.layers import Dense, Flatten, Reshape
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array, array_to_img
from keras.layers import UpSampling2D

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize pixel values to be between 0 and 1
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255
# Resize images to fit the input size required by VGG16 and ResNet50
x_train_resized = np.expand_dims(x_train, axis=-1)
x_train_resized = np.repeat(x_train_resized, 3, axis=-1)  # Make it three channels for VGG16 and ResNet50
x_test_resized = np.expand_dims(x_test, axis=-1)
x_test_resized = np.repeat(x_test_resized, 3, axis=-1)

# Convert labels to one-hot encoding
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Define base models
base_model_vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model_resnet50 = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Create model architecture
model_vgg16 = Sequential([
    UpSampling2D(size=(8, 8)),
    base_model_vgg16,
    Flatten(),
    Dense(256, activation='relu'),
    Dense(10, activation='softmax')
])

model_resnet50 = Sequential([
    UpSampling2D(size=(8, 8)),
    base_model_resnet50,
    Flatten(),
    Dense(256, activation='relu'),
    Dense(10, activation='softmax')
])

# Compile models
model_vgg16.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model_resnet50.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train models
history_vgg16 = model_vgg16.fit(x_train_resized, y_train, epochs=10, batch_size=128, validation_data=(x_test_resized, y_test))
history_resnet50 = model_resnet50.fit(x_train_resized, y_train, epochs=10, batch_size=128, validation_data=(x_test_resized, y_test))

# Plot accuracy and loss curves
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history_vgg16.history['accuracy'], label='VGG16 Training Accuracy')
plt.plot(history_vgg16.history['val_accuracy'], label='VGG16 Validation Accuracy')
plt.plot(history_resnet50.history['accuracy'], label='ResNet50 Training Accuracy')
plt.plot(history_resnet50.history['val_accuracy'], label='ResNet50 Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(history_vgg16.history['loss'], label='VGG16 Training Loss')
plt.plot(history_vgg16.history['val_loss'], label='VGG16 Validation Loss')
plt.plot(history_resnet50.history['loss'], label='ResNet50 Training Loss')
plt.plot(history_resnet50.history['val_loss'], label='ResNet50 Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')

plt.show()

# Report validation accuracy
print("VGG16 Validation Accuracy:", history_vgg16.history['val_accuracy'][-1])
print("ResNet50 Validation Accuracy:", history_resnet50.history['val_accuracy'][-1])

94765736/94765736 [==============================] - 1s 0us/step
Epoch 1/10
 13/469 [..............................] - ETA: 32:33:40 - loss: 5.5488 - accuracy: 0.0859